In [6]:
import pandas as pd

# Adjust the file path as necessary for your environment
file_path = "C:\\Users\\serge\\Downloads\\JASSTEST.xlsx"  # Update this to your actual file path
df = pd.read_excel(file_path, skiprows=[1])

# Convert 'Time' column to time format
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Find initial target index and value
initial_target_index = df[df['Close'] > 250].index.min()
initial_target_value = df.loc[initial_target_index, 'Close'] if pd.notnull(initial_target_index) else None

# Find index where time is 09:30:00
time_index = df[df['Time'] == pd.to_datetime('09:30:00').time()].index.min()

# Initialize columns
df['Target Value'] = pd.NA
df['Sell_Buy_CutPosition'] = None
df['Mark'] = None  
df['Loss'] = pd.NA

# Populate 'Target Value' and 'Sell_Buy_CutPosition' columns
if pd.notnull(initial_target_value) and pd.notnull(time_index):
    current_target_value = initial_target_value
    previous_target_value = initial_target_value
    for i in range(time_index, len(df)):
        df.at[i, 'Target Value'] = current_target_value
        if df.at[i, 'Close'] > current_target_value:
            if i != initial_target_index:
                loss = current_target_value - previous_target_value
                df.at[i, 'Loss'] = abs(loss)
            previous_target_value = current_target_value
            current_target_value = df.at[i, 'Close']
            df.at[i, 'Sell_Buy_CutPosition'] = 'Cut Position'
            df.loc[i:, 'Target Value'] = current_target_value
        else:
            df.at[i, 'Sell_Buy_CutPosition'] = 'Sell'
        if df.at[i, 'Time'] == pd.to_datetime('09:30:00').time():
            df.at[i, 'Sell_Buy_CutPosition'] = 'Dummy'

# Remove rows with null time values
df = df.dropna(subset=['Time'])

# Define function to mark 9:30 and 15:15 times
def mark_930(df):
    for idx, row in df.iterrows():
        if row['Time'] == pd.to_datetime('09:30:00').time():
            df.at[idx, 'Mark'] = 'N'
        elif row['Time'] == pd.to_datetime('15:15:00').time():
            df.at[idx, 'Mark'] = 'C'
    return df

# Apply marking function
df = mark_930(df)

# Mark 'C' for 'Cut Position' and adjust next row as 'N'
cut_position_indices = df[df['Sell_Buy_CutPosition'] == 'Cut Position'].index.tolist()
for idx in cut_position_indices:
    df.at[idx, 'Mark'] = 'C'
    if idx + 1 < len(df):
        df.at[idx + 1, 'Mark'] = 'N'

# Calculate difference and sum it up
df['Difference'] = None
last_n_close = None
last_n_index = None

for index, row in df.iterrows():
    if row['Time'] == '3:15':
        break
    if row['Mark'] == 'N':
        last_n_close = row['Close']
        last_n_index = index
    elif row['Mark'] == 'C' and last_n_close is not None:
        diff = last_n_close - row['Close']
        df.at[index, 'Difference'] = diff
        last_n_close = None
        last_n_index = None

total_difference = df['Difference'].sum()

# Set maximum number of rows displayed to None (show all rows)
pd.set_option('display.max_rows', None)

# Display selected columns
display_cols = ['Time', 'Close', 'Target Value', 'Sell_Buy_CutPosition', 'Difference']
print(df[display_cols])

# Display total difference
print("Total Difference:", total_difference)

         Time   Close Target Value Sell_Buy_CutPosition Difference
0    09:15:00  185.10         <NA>                 None       None
1    09:16:00  173.45         <NA>                 None       None
2    09:17:00  184.80         <NA>                 None       None
3    09:18:00  172.25         <NA>                 None       None
4    09:19:00  171.70         <NA>                 None       None
7    09:20:00  183.05         <NA>                 None       None
8    09:21:00  201.95         <NA>                 None       None
9    09:22:00  240.00         <NA>                 None       None
10   09:23:00  240.10         <NA>                 None       None
11   09:24:00  246.25         <NA>                 None       None
12   09:25:00  250.15         <NA>                 None       None
13   09:26:00  222.70         <NA>                 None       None
14   09:27:00  215.70         <NA>                 None       None
15   09:28:00  226.55         <NA>                 None       